In [10]:

!pip install IPython.Display

ERROR: Could not find a version that satisfies the requirement IPython.Display (from versions: none)
ERROR: No matching distribution found for IPython.Display


In [ ]:
#TODO
Additional Functionality
location tkinter





In [1]:
from opentrons import protocol_api
import tkinter as tk #basic python package apparently
import csv
import pandas as pd
import numpy as np
import os
from pandastable import Table, TableModel

In [2]:
## Meta Data

metadata = {'protocolName': 'RED',
'author':'Mason Kellinger masonkellinger@vt.edu',
 'description':'Restriction Enzyme Digestion',
 'apiLevel': '2.9'
}
print(metadata)

os.getcwd()

{'protocolName': 'RED', 'author': 'Mason Kellinger masonkellinger@vt.edu', 'description': 'Restriction Enzyme Digestion', 'apiLevel': '2.9'}


'C:\\Users\\mkell\\Documents\\GitHub\\opentrons\\Restriction'

In [3]:

#path = "/Shared drives/PlantSynBioLab/Opentrons/Restriction Enzyme Digestion/csvfiles"
path = "C:/Users/mkell/Desktop/RestrictionTestCSV2.csv"

#config_file = pd.read_csv(path, engine = 'python', encoding='utf-8-sig') ## last 2 parameters are error corrections
config_file = pd.read_csv("CommonEnzyme.csv",engine = 'python', encoding='utf-8-sig')

config_df = pd.DataFrame(config_file)

config_df

enzyme incu_temp incu_time inac_temp amount_dna
0  UNITS:   celsius   minutes   celsius         µg
1    SpeI        37        60        80          1
2   BbvcI        37        60       NaN          1
3    PmeI        37        60        65          1
4    SacI        37        60        65          1
5    NheI        37        60        65          1
6    BsaI        37        60        65          1
7    NotI        37        60        65          1

In [20]:
import tkinter as tk
from tkinter import *


enzyme_root = tk.Tk() #This creates the window
enzyme_root.title("Enzyme Parameter Selection")
enzyme_root.geometry("400x400")

def confirm1():
    global enzyme_name 
    enzyme_name = str(enzyme_entry.get())  #saves the first entry of the function to the variable enzyme_name
    
    global concentration_of_dna #same but for second entry
    concentration_of_dna = float(conc_entry.get())
    enzyme_root.destroy()
    

    
enzyme_label = tk.Label(text = "Enter Enzyme Name Here")
enzyme_label.place(relx=0.1,rely=0)

enzyme_entry = tk.Entry() #Creates the entry box
enzyme_entry.place(relx=.1,rely=.1)

conc_label = tk.Label(text="Enter your DNA concentration Here")
conc_label.place(relx=0.5, rely=0)

conc_entry = tk.Entry()
conc_entry.place(relx=0.6,rely = 0.1)

confirm_button = tk.Button(text="Confirm",command=confirm1) # extremely important to not have () at end of function
confirm_button.place(relx=0.45,rely=0.2)

name_label = tk.Label(text=str(config_df["enzyme"]))
name_label.place(relx = 0.05, rely = 0.2)

enzyme_root.mainloop() #This creates window loop, sensing for changes


NameError: name 'skip' is not defined

In [5]:
# maybe you could input the index of your enzyme as n and then iloc[n,1] ect.


incubation_temp = int(config_file.loc[config_file.enzyme==enzyme_name,"incu_temp"])
incubation_time = int(config_file.loc[config_file.enzyme==enzyme_name,"incu_time"])
inactivation_temp = int(config_file.loc[config_file.enzyme==enzyme_name,"inac_temp"])
amount_of_dna = int(config_file.loc[config_file.enzyme==enzyme_name,"amount_dna"])

volume_dna = amount_of_dna * (1/concentration_of_dna)
volume_water = 17 - volume_dna



In [16]:
config_df["enzyme"]

0    UNITS:
1      SpeI
2     BbvcI
3      PmeI
4      SacI
5      NheI
6      BsaI
7      NotI
Name: enzyme, dtype: object

1

In [14]:
from IPython.display import Image


Image(filename = "OpentronsDiagram.jpg",width=600, height=600)

In [1]:
# Protocol 

from opentrons import simulate
protocol = simulate.get_protocol_api('2.9')

    ## Loading labware
TempUnit = protocol.load_module('Temperature Module',3)
well = TempUnit.load_labware('opentrons_24_aluminumblock_nest_2ml_snapcap')
    
tiprack1 = protocol.load_labware("opentrons_96_tiprack_10ul",1)
tiprack2 = protocol.load_labware("opentrons_96_tiprack_20ul",2)
tuberack = protocol.load_labware("opentrons_24_tuberack_nest_2ml_snapcap",4)
    
        ## Loading tools
pipette_left = protocol.load_instrument('p20_single_gen2','left',tip_racks = [tiprack1])
pipette_right = protocol.load_instrument('p20_single_gen2','right',tip_racks = [tiprack2])
    
        ## Operations

    ## get tip -> get x amount of water to fill to 20uL -> dispense into tube
pipette_right.pick_up_tip(tiprack2['A1'])
pipette_right.aspirate(volume_water,tuberack['A1'],2)
pipette_right.dispense(volume_water,well['A1'],2) #create function to deteremine amount needed
pipette_right.blow_out()
pipette_right.drop_tip()

    ## get 2uL of 10x buffer -> dispense into tube -> get rid of tip

pipette_left.pick_up_tip()
pipette_left.aspirate(2,tuberack['A2'],2)
pipette_left.dispense(2,well['A1'],2)
pipette_left.blow_out()
pipette_left.drop_tip()


    ## get tip -> get y amount of DNA -> dispense into tube -> get rid of tip

pipette_left.pick_up_tip()
pipette_left.aspirate(volume_dna,tuberack['A3'],2) #create function to determine
pipette_left.dispense(volume_dna,well['A1'],2)
pipette_left.drop_tip()
    ## we don't blow out so we dont aerosolize dna

    ## get tip -> get 1uL Enzyme -> dispense into tube -> get rid of tip

pipette_left.pick_up_tip()  # mix to get all of enzyme
pipette_left.aspirate(1,tuberack['A4'],1)
pipette_left.dispense(1,well['A1'],1)
pipette_left.blow_out()
pipette_left.drop_tip()
    
    #mix
pipette_right.pick_up_tip()
pipette_right.mix(3,20,well['A1'])  #repetetions, max volume uL, location
    #this helps rinse the extra enzyme as well

## Temp Unit - incubate at temp for time t and then inactivate at temp for 10 min then rest at 4   degrees
                                        
TempUnit.set_temperature(incubation_temp)
protocol.delay(incubation_time)
TempUnit.set_temperature(inactivation_temp)
protocol.delay(10)
TempUnit.set_temperature(4)
    

C:\Users\mkell\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\mkell\.opentrons\deck_calibration.json not found. Loading defaults


NameError: name 'volume_water' is not defined